In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm
from scipy import stats
import json

In [120]:
def filter_data_new(data_in, max_step):
    """
    Only keeps data for tracks that are long enough for hurst exponent estimation at a given downsampling step size.
    Args: 
        data_in: pandas dataframe, original data
        max_step_size: int, maximum downsampling step size
        window: int, size of rolling window for hurst component estimation
    Returns:
        filtered_data: pandas dataframe
    """

    tracks_to_keep = data_in.TrackID.value_counts().loc[lambda x: x//max_step >2].reset_index()['index']

    filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep)]
    return filtered_data

def downsample(data_input, down_int, start_index=0):
    
    data_out = data_input.iloc[start_index::down_int]
   # print(data_out)
    return data_out



In [121]:
def get_mod(v):
    
    return np.sqrt(v.dot(v))

def get_step_vector(index,data_x,data_y,data_z):
    
    dx = data_x[index]-data_x[index-1]
    dy = data_y[index]-data_y[index-1]
    dz = data_z[index]-data_z[index-1]
    
    step_vector = np.array([dx,dy,dz])   

    return step_vector  

def get_ct(x_data,y_data,z_data):
    """
    iterates over a track (may be downsampled) and creates 1d array of cos(theta) values
    between each step
    """
    length=len(x_data)
    step_vectors = np.empty((length-1,3))
    cos_theta_temp = np.empty(length-2)
    for i in np.arange(length):
        if i>=1:
            step_vectors[i-1] = get_step_vector(i,x_data,y_data,z_data)
        if i>=2:
            v0 = step_vectors[i-2]
            v1 = step_vectors[i-1]
            cos_theta_temp[i-2] = np.dot(v0,v1)/(get_mod(v1)*get_mod(v0))
    return cos_theta_temp
    
def get_ct_ds_runs(track_data, ds_rate):
    """
    For a given track, applies downsampling and collects cos(theta) data for each 
    downsampling run (i.e. each of the 5 new tracks when downsampling an original track
    by 5) and appends the average of the cos theta track data to a 1d array
    """
    
    cos_theta_avgs_ds = np.array([])
    
    for i in np.arange(ds_rate):
        
        ds_track_data = downsample(track_data, ds_rate, start_index=i)
        
        x = ds_track_data['Position X'].values
        y = ds_track_data['Position Y'].values
        z = ds_track_data['Position Z'].values

        cos_theta_track = get_ct(x,y,z)
        
        avg_cos_theta_track = np.mean(cos_theta_track)
        
        cos_theta_avgs_ds = np.append(cos_theta_avgs_ds,avg_cos_theta_track)
    
    return cos_theta_avgs_ds
    
def get_ct_tracks(data_in,ds_rate,n_tracks=10000):
    
    cos_theta_tracks = np.array([])
    
    track_id_values = np.unique(data_in['TrackID'])
    
    for tid in track_id_values[:n_tracks]:

        track_data = data_in[data_in['TrackID']==tid]
        
        cos_theta_avg_ds = get_ct_ds_runs(track_data,ds_rate)
        
        cos_theta_tracks = np.append(cos_theta_tracks,cos_theta_avg_ds)
        
    return cos_theta_tracks
      
def get_cos_theta_dict(filtered_data, step_sizes,n_tracks=10000):
    """
    """
    cos_theta_dict = {}
    
    for i, s in enumerate(step_sizes):
        print('step_size ={}'.format(s))
        cos_theta_step = get_ct_tracks(filtered_data, s,n_tracks)
        cos_theta_dict["{}".format(s)] = cos_theta_step.tolist()
        
    return cos_theta_dict

def save_cos_theta_data(cos_theta_data, file_name,step_sizes,batch_number):#, window, step_sizes, restriction):
    """
    """
    with open('cos_theta_dict_'+file_name+'_s{}'.format(max(step_sizes))+'_b{}'.format(batch_number), 'w') as file:
        file.write(json.dumps(cos_theta_data))
      

In [122]:
filenames = np.array(['Control_frame001-200',
                      'Control_frame200-400',
                      'Control_frame400-600',
                      'Control_frame600-800',
                      'Control_frame800-1000',
                      'Control_frame1000-1200',
                      'LanB1_frame001-200',
                      'LanB1_frame200-400',
                      'LanB1_frame400-600',
                      'LanB1_frame600-800',
                      'LanB1_frame800-1000',
                      'LanB1_frame1000-1200',
                      'defLanB1_300817_frame200-400',
                      'defLanB1_300817_frame400-600'])

In [123]:
step_sizes = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
n_tracks_restricted = 10000

# iterate over data files and save H dictionary 
for i, file in enumerate(filenames):
    print('opened file {}'.format(file))
    data = pd.read_csv('haemocyte_tracking_data/' + file + '.csv')
    filtered_data = filter_data_new(data,max(step_sizes))
    cos_theta_dict = get_cos_theta_dict(filtered_data, step_sizes,n_tracks_restricted)
    #print(cos_theta_dict)
    save_cos_theta_data(cos_theta_dict, file,step_sizes,1)

opened file Control_frame001-200
step_size =1
step_size =2
step_size =3
step_size =4
step_size =5
step_size =6
step_size =7
step_size =8
step_size =9
step_size =10
step_size =11
step_size =12
step_size =13
step_size =14
step_size =15
opened file Control_frame200-400
step_size =1
step_size =2
step_size =3
step_size =4
step_size =5
step_size =6
step_size =7
step_size =8
step_size =9
step_size =10
step_size =11
step_size =12
step_size =13
step_size =14
step_size =15
opened file Control_frame400-600
step_size =1
step_size =2
step_size =3
step_size =4
step_size =5
step_size =6
step_size =7
step_size =8
step_size =9
step_size =10
step_size =11
step_size =12
step_size =13
step_size =14
step_size =15
opened file Control_frame600-800
step_size =1
step_size =2
step_size =3
step_size =4
step_size =5
step_size =6
step_size =7
step_size =8
step_size =9
step_size =10
step_size =11
step_size =12
step_size =13
step_size =14
step_size =15
opened file Control_frame800-1000
step_size =1
step_size =2
step

## SCRAP

In [6]:

    #if restriction<len(tracks_to_keep):
     #   filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep[:restriction])]
    #else:
     #   filtered_data = data_in[data_in['TrackID'].isin(tracks_to_keep)]

def get_cos_theta(data_in,restriction):
    
    track_id_values = np.unique(data_in['TrackID'])
    cos_theta_avg = np.array([])
    
    print('------------')
    
    for tid in track_id_values[:restriction]:
        track_data = data_in[data_in['TrackID']==tid]
        x = track_data['Position X'].values
        y = track_data['Position Y'].values
        z = track_data['Position Z'].values
        
        cos_theta_temp = get_ct(x,y,z)

        cos_theta_avg = np.append(cos_theta_avg,np.mean(cos_theta_temp))
        
    return cos_theta_avg

def dsample_get_cos_theta(data_in, ds_rate,restriction):
    """
    Args:
        data_in: pandas dataframe
        ds_rate: int, downsampling step size
    Returns:
        h_arr: 2D np array
    """
    #cos_theta_avgs = np.array([])
    for i in np.arange(ds_rate):
        
        downsampled_data = downsample(data_in, ds_rate, start_index=i)

        cos_theta_avgs_temp = get_cos_theta(downsampled_data,restriction)
        print(len(cos_theta_avgs_temp))
            
        if i == 0:
            cos_theta_avgs = cos_theta_avgs_temp
        else:
            
            if len(cos_theta_avgs_temp)>len(cos_theta_avgs):
                cos_theta_avgs_temp=cos_theta_avgs_temp[:-1]
            
            cos_theta_avgs = np.append(cos_theta_avgs,cos_theta_avgs_temp)
        
    return cos_theta_avgs
